In [ ]:
import os
import shutil
import zipfile

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def download_zip(url, output_path):
    response = requests.get(url, headers=headers, stream=True)
    if response.status_code == 200:
        with open(output_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"ZIP file downloaded to {output_path}")
    else:
        print(f"Failed to download. HTTP Response Code: {response.status_code}")

url = "http://www.manythings.org/anki/fra-eng.zip"
output_path = "fra-eng.zip"
download_zip(url, output_path)

path = os.getcwd()
zipfilename = os.path.join(path, output_path)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

# 영어와 프랑스어의 병렬 데이터를 받아옴.

ZIP file downloaded to fra-eng.zip


In [ ]:
# fra.txt 파일을 탭을 구분자로 하여 읽어들이고, 구분자로 구분되어 생성된 각 행의 열을 'src', 'tar', 'lic' 이름의 행에 저장함.
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
print('전체 샘플의 개수 :',len(lines))
# 필요한 정보인 source와 target data만 남기고 lic 열은 삭제해줌.

전체 샘플의 개수 : 229803


읽어들인 데이터들 중에서 'src와 'tar'열의 데이터만을 다시 저장해주고,
전체 대이터 23만개 중에서 6만개의 데이터만을 사용해 학습을 진행한다.

In [ ]:
lines = lines.loc[:, 'src':'tar']
lines = lines[:100000]
lines.sample(10)
lines.head(10)

,src,tar
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !
5,Hi.,Salut.
6,Run!,Cours !
7,Run!,Courez !
8,Run!,Prenez vos jambes à vos cous !
9,Run!,File !


아래 코드는 pandas의 열 데이터에 대한 변환을 수행하는 코드이다. lines.tar를 사용하여 tar 열 데이터를 참조할 수 있도록 했고, lines.tar.apply(function)을 사용하여 각 데이터를 변환하여 적용해주었다.

lambda x는 재참조를 하지 않을 함수를 간단하게 생성하는 함수이다.

```python
lambda x : '\t '+ x + ' \n'
```
따라서 위의 코드는 데이터의 가장 앞과 가장 뒤에 데이터를 추가하는 함수를 구현한 것이다.

In [ ]:
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(10)

,src,tar
45862,You don't want that.,\t Tu ne veux pas cela. \n
5255,I hate flies.,\t Je déteste les mouches. \n
33256,I can't wait to go.,\t J'ai hâte d'y aller. \n
52508,Tom is always crying.,\t Tom pleure tout le temps. \n
11676,I want to live.,\t Je veux vivre. \n
35456,My family is small.,\t Ma famille est petite. \n
50725,Look at this picture.,\t Regardez cette photo. \n
44034,This guy is a loser.,\t Ce mec est un nullard. \n
75380,I've told you the truth.,\t Je vous ai dit la vérité. \n
72934,How long will this last?,\t Combien de temps ceci durera-t-il ? \n


  문자 집합을 구축하기 위해서 set 자료형을 사용하는 것을 확인할 수 있다. Set 자료형의 경우 중복되는 값은 저장할 수 없기에 char data set을 생성할 때 활용할 수 있다.

  lines.src와 lines.tar 코드를 활용하여 각 열에 대한 정보를 행 단위로 받는 것이고,
  for char in line 코드를 문자열 line에 사용하여 각 문자로 자동적인 parsing이 가능하도록 하였다.

In [ ]:
# 문자 집합 구축
src_vocab = set()
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 문자씩 읽음
        src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)


In [ ]:
src_vocab_size = len(src_vocab) + 1
tar_vocab_size = len(tar_vocab) + 1
print('source 문장의 char 집합 :',src_vocab_size)
print('target 문장의 char 집합 :',tar_vocab_size)

source 문장의 char 집합 : 82
target 문장의 char 집합 : 106


src_vocab와 tar_vocab 집합을 임의로 정렬하고, 일부를 출력해준다.

set 자료형은 정렬 sorted(set()) 형태로 사용하지 않는다면 인덱스 슬라이싱을 사용할 수 없기에 sorted 명령어를 사용하여 정렬한 후에, 해당 값을 출력해주어야 한다.

In [ ]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w']


enumerate() 함수의 경우, 데이터의 인덱스와 값을 반환해주는 역할을 한다.
따라서 사전의 형태로 각 word에 대응되는 인덱스를 저장해주는 코드이다.

해당 dictionary를 활용하여 문자열을 정수 데이터로 전환하는 과정에 사용할 수 있을 것이다.

In [ ]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, '\xa0': 76, '°': 77, 'é': 78, 'ï': 79, '’': 80, '€': 81}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F'

lines.src에서 line에 해당하는 source값을 하나씩 받아온다.

```python
for char in line
```
위 구문을 통해서 문자열 parsing을 사용하여 character를 하나씩 받아오고
```python
encoded_line.append(src_to_index[char])
```
위 코드를 사용하여 미리 선언한 list에 각 문자열에 대응되는 숫자를 append 시켜준다.
append의 경우, list의 차원을 증가시키지 않으므로, 1차원 리스트 안에서 여러개의 정수가 존재하는  형태가 될 것이다.

```python
encoder_input.append(encoded_line)
```
위 코드를 사용하여 encoder_input이라는 저장 리스트 안에 리스트를 append 시켜준다.

for문을 사용하여 계속해서 1차원 리스트를 리스트 안에 append시켜주므로, 결국 2차원 리스트 형태로 저장이 될 것이다.

## 저장 형태

저장 형태를 잘 살펴보면 2차원 리스트라는 것은 쉽게 파악이 가능하다.

세부적으로 살펴보자면, 2차원 리스트의 하위 리스트는 한 line을 통째로 바꾸는 모습을 확인할 수 있다.

In [ ]:
encoder_input = []

# 1개의 문장
for line in lines.src:
  encoded_line = []
  # 각 줄에서 1개의 char
  for char in line:
    # 각 char을 정수로 변환
    encoded_line.append(src_to_index[char])
  encoder_input.append(encoded_line)
print('source 문장의 정수 인코딩 :',encoder_input[:5])

source 문장의 정수 인코딩 : [[30, 64, 10], [30, 64, 10], [30, 64, 10], [30, 64, 10], [31, 58, 10]]


In [ ]:
decoder_input = []
for line in lines.tar:
  encoded_line = []
  for char in line:
    encoded_line.append(tar_to_index[char])
  decoder_input.append(encoded_line)
print('target 문장의 정수 인코딩 :',decoder_input[:5])


target 문장의 정수 인코딩 : [[1, 3, 48, 53, 3, 4, 3, 2], [1, 3, 39, 53, 70, 55, 60, 57, 14, 3, 2], [1, 3, 31, 66, 3, 70, 67, 73, 72, 57, 3, 4, 3, 2], [1, 3, 28, 67, 73, 59, 57, 3, 4, 3, 2], [1, 3, 45, 53, 64, 73, 72, 3, 4, 3, 2]]


In [ ]:
decoder_target = []
for line in lines.tar:
  timestep = 0
  encoded_line = []
  for char in line:
    if timestep > 0:
      encoded_line.append(tar_to_index[char])
    timestep = timestep + 1
  decoder_target.append(encoded_line)
print('target 문장 레이블의 정수 인코딩 :',decoder_target[:5])

target 문장 레이블의 정수 인코딩 : [[3, 48, 53, 3, 4, 3, 2], [3, 39, 53, 70, 55, 60, 57, 14, 3, 2], [3, 31, 66, 3, 70, 67, 73, 72, 57, 3, 4, 3, 2], [3, 28, 67, 73, 59, 57, 3, 4, 3, 2], [3, 45, 53, 64, 73, 72, 3, 4, 3, 2]]


In [ ]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('source 문장의 최대 길이 :',max_src_len)
print('target 문장의 최대 길이 :',max_tar_len)
# 문장의 길이를 전부 동일하게 맞추기 위해서 max 함수를 사용함.

source 문장의 최대 길이 : 27
target 문장의 최대 길이 : 76


In [ ]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')
print('source 문장의 정수 인코딩 :',encoder_input[:10])
print('target 문장의 정수 인코딩 :',decoder_input[:10])
print('target 문장 레이블의 정수 인코딩 :',decoder_target[:10])
# 문장 길이를 동일하게 만들어주기 위해 padding을 사용함.
# 최대 입력 소스와 최대 길이, padding 옵션에는 post를 사용했으므로 원래 데이터 이후의 값이라면 0이 들어간다.

source 문장의 정수 인코딩 : [[30 64 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [30 64 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [30 64 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [30 64 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [31 58 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [31 58 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [41 70 63  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [41 70 63  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [41 70 63  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [41 70 63  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]]
target 문장의 정수 인코딩 : [[  1   3  48  53   3   4   3   2   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0

단어 단위 번역기로 사용하는 경우, Embedding Layer를 사용하여 워드 임베딩을 사용해야 하지만. 이 경우 문자 단위 번역기에 해당하므로 Embedding Layer를 사용하지 않아도 된다.

띠라서 one hot vector를 사용한다.

In [ ]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)
print('source 문장의 one hot 인코딩 :',encoder_input[:10])
print('target 문장의 one hot 인코딩 :',decoder_input[:10])
print('target 문장 레이블의 one hot 인코딩 :',decoder_target[:10])
print(encoder_input.shape)
print(decoder_input.shape)
print(decoder_target.shape)

source 문장의 one hot 인코딩 : [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. .

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np
if tf.config.list_physical_devices('GPU'):
  tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)

## **Encoder Input**

Tensorflow Input을 사용하여 입력 시퀀스의 형태를 정의한다.
shape를 (None, src_vocab_size)를 사용하여 각 시퀀스의 크기를 지정해준다.

Input layer에 0ne-hot encoded input이 들어간다. 또한, encoder에 들어가는 source의 크기는 각 단어별로 src_vocab_size 길이의 리스트 형태를 가져야 하므로 해당 shape를 사용하는 것으로 생각할 수 있다.

## **Encoder LSTM**

우리가 Seq2Seq에 사용할 RNN Unit을 지정하는 것이라고 생각할 수 있다. 이번 예제에서는 LSTM 유닛을 사용하여 RNN을 구현할 것이기에, 이를 사용하는 코드를 작성하는 것이고, return_state = True의 코드를 사용하여 최종 state를 반환하도록 했다. 이 최종 state는 Decoder에서 사용된다.

<span style="background-color: #FFFF00"> LSTM의 구조에 관해서 더 공부해보아야 한다. 왜 256unit이라는 것을 사용하는지 다시 공부해야겠다. </span>

LSTM의 구조에 관해서 더 자세히 공부해볼 예정이지만, unit의 크기라는 것은 쉽게 말해서 hidden state의 크기를 결정하는 인자라고 볼 수 있다. 256unit이라는 것은 hidden state가 256개의 항의 개수, 256차원을 가진다라고 생각하면 된다.

```python
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
```
위 코드를 사용하여 one hot encoded data가 LSTM 레이어를 통과한 이후의 결과값을 저장할 수 있다. LSTM 레이어를 통과한 이후의 결과값은 3개가 존재하는데, 먼저 output, hidden state, cell state이다.

다른 부분들은 쉽게 이해되지만, **cell state에 대한 부분이 제대로 이해되지 않는다.** 여기에서는 간단하게 LSTM의 장기적인 의존성을 관리하는 데에 사용한다라고 이해할 수 있다.

**결과적으로 hidden state와 cell state를 사용하여 encoder의 최종 state를 생성하고(list 형태로) 이를 활용하여 Decoder의 Input값을 생성할 수 있다.**

In [ ]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)

# encoder_outputs은 여기서는 불필요
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 은닉 상태와 셀 상태.
encoder_states = [state_h, state_c]

## **Decoder Input**

decoder input은 tensorflow의 Input을 사용하여 encoder의 input과 동일하게, one hot encoded data의 각 문자의 크기(변주범위)를 제공하는 것으로 정의된다.

## **Decoder LSTM**

encoder에서 RNN 기반의 LSTM cell이 사용되었으므로, decoder에서도 Lstm이 사용되어야 한다. 따라서 tensorflow의 LSTM() 함수를 사용하여 LSTM 레이어를 작성해주었다.

return_sequences 옵션을 사용하여 모든 레이어에 대한 출력을 시퀀스로 반환해준다.

* return_sequences = True

  return sequence 옵션이 True로 설정된다면, 매 시점의 출력을 저장하여 (batch_size, sequence_length, units)의 형태로 출력값을 지니게 된다. Decoder에서 이 옵션을 true로 설정한다면 매 시점의 정보 형태를 유지한다는 의미이므로 Seq2Seq 모델이나 시계열 데이터 예측 등에 사용된다.

* return_sequences = False

  return sequence 옵션이 False로 설정된다면, 매 시점의 정보를 저장하는 것이 아니라 마지막 하나의 출력만을 저장하는 RNN 유닛이 생성된다. 이 경우 전체 데이터를 활용하여 하나의 정보로 압축하는 형태로 사용하는 것이기에, 긍정/부정이나 카테고리 분류 등등에 사용할 수 있다. 이 옵션은 출력의 형태가 (batch_size, units)의 형태이다.

우리는 Seq2Seq 모델에 대해서 공부하고 있으므로, 출력의 형태를 유지해야 하며 이 옵션은 True로 설정해두어야 한다.

****
추가적으로, Lstm의 initial state에 해당하는 값으로 encoder_states를 사용하는 모습을 확인할 수 있는데, 이는 Encoder의 최종 출력층에 해당하는 Context Vector를 initial state로 활용하는 모습이다.

## Decoder Output

Decoder의 동작을 생각해보자. 인코더에서 받은 Context Vector를 바탕으로 출력값을 생성해내는 역할을 하는데, 이 특성을 이해한다면, LSTM의 출력값에 해당하는 output, hidden state, cell state중에서 hidden, cell state는 필요하지 않다는 것을 생각할 수 있다.

해당 처리를 거쳐서 나온 Output에 대해서 생각해보자.decoder_outputs의 데이터를 Dense layer를 만들어 해당 레이어를 통과시키는 모습을 볼 수 있다. Dense Layer는 pytorch의 nn.Linear와 같은 FC Layer에 해당한다고 생각하면 된다. 선형 변환과 같은 Full connected layer를 정의한 뒤, 해당 결과값을 바탕으로 softmax를 사용하여 각 단어별 확률을 계산하는 방식을 사용한다.

<span style="color:blue">**여기서 왜 activation function 이전에 Fully connected layer를 붙여주어야 하는지에 대한 의문점이 생겼다.**</span>

이는 다른 post에서 정리하겠다.

## **Model 훈련**

Encoder의 hidden state, cell state, Decoder의 output을 받는 Model을 구성한다.

Pytorch 에서는 optimizer로 구현된 것과 critieon으로 표현하는 loss값을 comile 함수를 통해서 지정해준다. 이후 모델의 validation 분할 비율과 배치 사이즈 등등을 결정해 모델 피팅을 진행한다.

In [ ]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)

# 디코더에게 인코더의 은닉 상태, 셀 상태를 전달.
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [ ]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=2048, epochs=400, validation_split=0.2)

Epoch 1/400
40/40 [==============================] - 16s 303ms/step - loss: 1.8627 - val_loss: 1.8056
Epoch 2/400
40/40 [==============================] - 8s 202ms/step - loss: 1.2851 - val_loss: 1.4231
Epoch 3/400
40/40 [==============================] - 8s 199ms/step - loss: 1.1586 - val_loss: 1.3593
Epoch 4/400
40/40 [==============================] - 8s 202ms/step - loss: 1.0881 - val_loss: 1.3415
Epoch 5/400
40/40 [==============================] - 8s 203ms/step - loss: 1.0328 - val_loss: 1.2631
Epoch 6/400
40/40 [==============================] - 8s 204ms/step - loss: 0.9685 - val_loss: 1.2279
Epoch 7/400
40/40 [==============================] - 8s 208ms/step - loss: 0.9196 - val_loss: 1.1424
Epoch 8/400
40/40 [==============================] - 8s 208ms/step - loss: 0.8703 - val_loss: 1.0723
Epoch 9/400
40/40 [==============================] - 8s 210ms/step - loss: 0.8290 - val_loss: 1.0398
Epoch 10/400
40/40 [==============================] - 8s 211ms/step - loss: 0.7976 - val_l

In [ ]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

## Test 과정에서 필요한 디코더 설계

Test 과정에서 사용하는 디코더의 경우, First Cell에는 Context Vector와 SOS 신호가 들어가며, 두번째 Cell부터는 이전 cell의 hidden state와 cell state, 그리고 output이 현재 cell의 입력값으로 활용된다.

따라서 이를 위해서 필요한 것들은 아래와 같다.

* Context Vector - Encoder_states
* decoder hidden state - state_h
* decoder cell state - state_c

Input layer가 256이라는 것은, LSTM이 256 크기의 hidden state와 cell state를 사용하기 에 설정되는 값이다. 결국, Layer를 설계하는 과정이라고 보면 된다. (실제 값이 아닌 정의)

따라서, decoder에 입력되는 state들을 사전에 정의해 놓은 것이 decoder_states_inputs이다.

입력 Layer를 설정했다면, 이제 LSTM Layer를 정의해야 한다. LSTM Layer의 정의를 위해선 여러가지 옵션이 존재하는데, training 단계에서 정의한 것과 동일한 옵션을 사용해야 한다.

```python
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
```

따라서 decoder_lstm의 정의 과정이 생략되었지만, LSTM 설정이 완료되었으므로, LSTM의 input과 Input Layer의 Output을 연결시켜주어야 한다. LSTM의 구조에서 입력값으로 Hidden state와 input값이 사용되는 것은 알고 있다. 이를 위해서, 입력값을 사전에 정의해둔 decoder_inputs를 사용하여 형태를 잡는 것이고, hidden state는 최초의 hidden state만을 decoder_state_inputs로 사용하여 형태를 잡은 것이다.

그렇다면, 최초의 cell을 제외한 나머지 형태에 대해서 이어주는 코드를 사용해야 한다. 이를 위해서
```python
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
```
위의 코드를 사용하게 되는데,
lstm Layer의 결과값에 해당하는 output, hidden state, cell state를 전부 받아서 이를 다음 cell에 연결해주어야 한다.

```python
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)
```
해당 구현은 Model 안에 input과 output을 정의하는 것으로 사용할 수 있다.

Model을 정의하는 부분을 잘 살펴보아야 한다.

Model의 인자에 넘겨주는 Input과 Output은 각각 모델이 처리할 **초기 데이터와 최종 데이터** 형태를 나타내는 것에 불과하고, 실제 세부적인 구현은 Input()이나 Dense() 등의 명령어를 통해서 구현하는 것이다.

Test 단계에서는 Model을 통해 Decoder를 구현해야 한다. 따라서 Context Vector와 최초의 input(SOS)를 받고, 최종 출력층에서는 hidden state와 Output에 해당하는 값을 받는 model을 정의한다.

**따라서 최종적인 구성은 Model에 input과 output값을 정의해주는 것으로 마무리되며, model.fit()이나 model.predict() 등등의 명령어에 의해서 input값으로 어떤 값을 사용할지 자동적으로 프레임워크수준에서 결정되게 된다.**


In [ ]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

## InttoText

앞선 과정에서 dictionary 자료형을 활용하여 문자를 key로, 정수를 value로 하는 번역기를 생성했는데, Test 과정에서 나온 output을 사용하여 다시 문자로 변환하기 위해서는 정수를 key로, 문자를 value로 하는 번역기(Dictionary)를 정의해야 한다.

해당 코드는 .items()를 통해 값을 전부 반환하는 코드를 통해서 받아 반대로 dictionary를 생성하는 코드를 작성할 수 있다.

In [ ]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

아래 decode_sequence 함수의 동작 과정은 다음과 같다.

1. encoder model에서 context vector를 추출한다.
2. decoder model의 입력 sequence에 해당하는 <sos>신호를 만들어낸다.이는 특정 자리에 1의 값을 대입하는 방식으로 이루어진다.
3. SOS 대입 이후 생성된 데이터를 이용하여 정답지 Index를 생성하고, 해당 인덱스를 바탕으로 다음 시점의 입력과 정답지를 생성한다.
4. 위 과정을 반복하여 정답지에 EOS 표지가 나오거나 최대 길이에 도달한다면 이를 종료하고 값을 반환한다.

model.predict()의 입력에 해당 정답지를 사용한다는 것을 통해서 LSTM 등의 구조에서 hidden state는 자동적으로 관리되고, input의 값에만 집중한다는 사실을 확인할 수 있었다.

In [ ]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 원-핫 벡터 생성
  target_seq = np.zeros((1, 1, tar_vocab_size))
  target_seq[0, 0, tar_to_index['\t']] = 1.

  stop_condition = False
  decoded_sentence = ""

  # stop_condition이 True가 될 때까지 루프 반복
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 문자로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 문자를 예측 문장에 추가
    decoded_sentence += sampled_char

    # <eos>에 도달하거나 최대 길이를 넘으면 중단.
    if (sampled_char == '\n' or
        len(decoded_sentence) > max_tar_len):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, sampled_token_index] = 1.

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence


In [ ]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
  input_seq = encoder_input[seq_index:seq_index+1]
  decoded_sentence = decode_sequence(input_seq)
  print(35 * "-")
  print('입력 문장:', lines.src[seq_index])
  print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
  print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력


1/1 [==============================] - 0s 19ms/step
-----------------------------------
입력 문장: Go.
정답 문장: Bouge ! 
번역 문장: Autrends ! 
1/1 [==============================] - 0s 20ms/step
-----------------------------------
입력 문장: Hello!
정답 문장: Bonjour ! 
번역 문장: Ainez-nous ! 
1/1 [==============================] - 0s 19ms/step
-----------------------------------
입력 문장: Got it!
정답 문장: J'ai pigé ! 
번역 문장: Aidez Tom. 
1/1 [==============================] - 0s 20ms/step
-----------------------------------
입력 문장: Go home.
정답 문장: Rentre à la maison. 
번역 문장: Va tour le choix. 
1/1 [==============================] - 0s 20ms/step
-----------------------------------
입력 문장: Get going.
정답 문장: En avant. 
번역 문장: Décure. 
